<a href="https://colab.research.google.com/github/egorneo/LLM-techniques/blob/main/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plain linear quantizer

$r = s(q-z)$
Where r is initial tensor(its component), q - quantized tensor, s - some multiplier, z - zero point.

$\frac{r}{s} = q - z$





In [1]:
import torch

In [49]:
def linear_quantizer(tensor, s, z):
  tensor = tensor/s + z
  tensor = torch.round(tensor)
  print(tensor)
  tensor = torch.clamp(tensor, min = -128, max = 127)
  return tensor.to(torch.int8)

dummy_tensor = torch.tensor(
    [[191.6, -13.5, 728.6],
     [92.14, 295.5,  -184],
     [0,     684.6, 245.5]]
)
print(dummy_tensor)
quantized = linear_quantizer(dummy_tensor, 3.5, -70)

tensor([[ 191.6000,  -13.5000,  728.6000],
        [  92.1400,  295.5000, -184.0000],
        [   0.0000,  684.6000,  245.5000]])
tensor([[ -15.,  -74.,  138.],
        [ -44.,   14., -123.],
        [ -70.,  126.,    0.]])


In [51]:
def linear_dequantizer(tensor, s, z):
  return s*(tensor.float() - z)

dequantized = linear_dequantizer(quantized, 3.5, -70)
print(f"Initial tensor: \n {dummy_tensor} \n \nDequantized tensor: \n {dequantized}")

torch.norm(dummy_tensor - dequantized)

Initial tensor: 
 tensor([[ 191.6000,  -13.5000,  728.6000],
        [  92.1400,  295.5000, -184.0000],
        [   0.0000,  684.6000,  245.5000]]) 
 
Dequantized tensor: 
 tensor([[ 192.5000,  -14.0000,  689.5000],
        [  91.0000,  294.0000, -185.5000],
        [   0.0000,  686.0000,  245.0000]])


tensor(39.2158)

Naive approach with max

In [47]:
norm = dummy_tensor.abs().max()
print(norm)
quantized = torch.round(dummy_tensor/norm*(255)).to(torch.int8)
print(quantized)
dequantized = quantized.float()/255*norm
print(dequantized)
torch.norm(dummy_tensor - dequantized)

tensor(728.6000)
tensor([[ 67,  -5,  -1],
        [ 32, 103, -64],
        [  0, -16,  86]], dtype=torch.int8)
tensor([[ 191.4361,  -14.2863,   -2.8573],
        [  91.4322,  294.2972, -182.8643],
        [   0.0000,  -45.7161,  245.7239]])


tensor(1033.6320)